In [24]:
import pandas as pd
import numpy as np
import pyspark
import pyspark.sql.functions as sf

In [3]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/20 14:40:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/20 14:40:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


1. Create a spark data frame that contains your favorite programming languages.
    - The name of the column should be `language`

In [2]:
x = {'language': ['python', 'pandas', 'spark', 'ruby', 'java', 'javascript']}

In [5]:
x = pd.DataFrame(x)

In [6]:
df = spark.createDataFrame(x)

In [10]:
df.show()

+----------+
|  language|
+----------+
|    python|
|    pandas|
|     spark|
|      ruby|
|      java|
|javascript|
+----------+



    - View the schema of the dataframe

In [11]:
df.printSchema()

root
 |-- language: string (nullable = true)



    - Output the shape of the dataframe

In [19]:
df.count()

6

    - Show the first 5 records in the dataframe

In [20]:
df.show(5)

+--------+
|language|
+--------+
|  python|
|  pandas|
|   spark|
|    ruby|
|    java|
+--------+
only showing top 5 rows



2. Load the `mpg` dataset as a spark dataframe.

In [21]:
from pydataset import data

In [22]:
mpg = spark.createDataFrame(data('mpg'))

In [23]:
mpg.show(1)

+------------+-----+-----+----+---+--------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|   trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+--------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|auto(l5)|  f| 18| 29|  p|compact|
+------------+-----+-----+----+---+--------+---+---+---+---+-------+
only showing top 1 row



    1. Create 1 column of output that contains a message like the one below:

            The 1999 audi a4 has a 4 cylinder engine.

        For each vehicle.

In [36]:
textdf = mpg.select(sf.concat(sf.lit("The "),
                  mpg.year,
                  sf.lit(" "),
                  mpg.manufacturer,
                  sf.lit(" has a "),
                  mpg.cyl, 
                  sf.lit(" cylinder engine.")
                 ).alias('output')
          )
textdf.show(truncate=False)

+-------------------------------------------+
|output                                     |
+-------------------------------------------+
|The 1999 audi has a 4 cylinder engine.     |
|The 1999 audi has a 4 cylinder engine.     |
|The 2008 audi has a 4 cylinder engine.     |
|The 2008 audi has a 4 cylinder engine.     |
|The 1999 audi has a 6 cylinder engine.     |
|The 1999 audi has a 6 cylinder engine.     |
|The 2008 audi has a 6 cylinder engine.     |
|The 1999 audi has a 4 cylinder engine.     |
|The 1999 audi has a 4 cylinder engine.     |
|The 2008 audi has a 4 cylinder engine.     |
|The 2008 audi has a 4 cylinder engine.     |
|The 1999 audi has a 6 cylinder engine.     |
|The 1999 audi has a 6 cylinder engine.     |
|The 2008 audi has a 6 cylinder engine.     |
|The 2008 audi has a 6 cylinder engine.     |
|The 1999 audi has a 6 cylinder engine.     |
|The 2008 audi has a 6 cylinder engine.     |
|The 2008 audi has a 8 cylinder engine.     |
|The 2008 chevrolet has a 8 cylind

    1. Transform the `trans` column so that it only contains either `manual` or `auto`.

In [56]:
mpg.select(sf.regexp_replace("trans", r"\(.+\)", "").alias("trans")).show(5)

+------+
| trans|
+------+
|  auto|
|manual|
|manual|
|  auto|
|  auto|
+------+
only showing top 5 rows



3. Load the `tips` dataset as a spark dataframe.

In [57]:
tips = spark.createDataFrame(data('tips'))

In [58]:
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



    1. What percentage of observations are smokers?

In [77]:
tips.describe().show()

+-------+-----------------+-----------------+------+------+----+------+------------------+
|summary|       total_bill|              tip|   sex|smoker| day|  time|              size|
+-------+-----------------+-----------------+------+------+----+------+------------------+
|  count|              244|              244|   244|   244| 244|   244|               244|
|   mean|19.78594262295082| 2.99827868852459|  null|  null|null|  null| 2.569672131147541|
| stddev|8.902411954856857|1.383638189001182|  null|  null|null|  null|0.9510998047322344|
|    min|             3.07|              1.0|Female|    No| Fri|Dinner|                 1|
|    max|            50.81|             10.0|  Male|   Yes|Thur| Lunch|                 6|
+-------+-----------------+-----------------+------+------+----+------+------------------+



In [107]:
# tips.select(((sf.count(tips.smoker != sf.lit('No'))
#              / sf.count(tips.smoker)) * 100).alias('prct_smokers')).show()

s = tips.groupBy('smoker').agg(sf.count(tips.smoker).alias('smoker_counts'))
s.show()

+------+-------------+
|smoker|smoker_counts|
+------+-------------+
|    No|          151|
|   Yes|           93|
+------+-------------+



In [111]:
# s.select((s.filter(s.smoker == sf.lit('Yes')).smoker_counts) / sf.sum(s.smoker_counts)).show()

In [114]:
s.createOrReplaceTempView("s")

In [118]:
spark.sql("SELECT smoker_counts FROM s WHERE smoker = 'Yes'").show()

+-------------+
|smoker_counts|
+-------------+
|           93|
+-------------+



In [120]:
spark.sql(
    """
    SELECT ROUND((SELECT smoker_counts FROM s WHERE smoker = 'Yes')/sum(smoker_counts)*100, 2) as smoker_prct
    FROM s
    """
).show()

+-----------+
|smoker_prct|
+-----------+
|      38.11|
+-----------+



    1. Create a column that contains the tip percentage

In [121]:
tips.show(1)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
+----------+----+------+------+---+------+----+
only showing top 1 row



In [134]:
col = tips.select((sf.expr('tip / total_bill * 100')))
col

DataFrame[((tip / total_bill) * 100): double]

In [138]:
tips = tips.withColumn('tip_prct', sf.expr('tip / total_bill * 100'))
tips.show(5)

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|          tip_prct|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|5.9446733372572105|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|16.054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|16.658733936220845|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 13.97804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|14.680764538430255|
+----------+----+------+------+---+------+----+------------------+
only showing top 5 rows



    1. Calculate the average tip percentage for each combination of sex and smoker.

In [137]:
tips.groupby('sex').pivot('smoker').mean('tip_prct').show()

+------+------------------+------------------+
|   sex|                No|               Yes|
+------+------------------+------------------+
|Female| 15.69209707691836| 18.21503526994103|
|  Male|16.066871512912982|15.277117520248513|
+------+------------------+------------------+



In [190]:
# another way
tips.groupBy("sex", "smoker").agg(sf.round(
        sf.mean("tip_prct"), 4)
         .alias("tip_pct")
    ).show()

+------+------+-------+
|   sex|smoker|tip_pct|
+------+------+-------+
|  Male|    No|16.0669|
|Female|    No|15.6921|
|  Male|   Yes|15.2771|
|Female|   Yes| 18.215|
+------+------+-------+



4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [192]:
from vega_datasets import data
weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(2)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 2 rows



   - Convert the temperatures to fahrenheit.

In [193]:
weather = weather.withColumn('temp_max', sf.expr('ROUND((((9/5)*temp_max) + 32),1)'))\
.withColumn('temp_min', sf.expr('ROUND((((9/5)*temp_min) + 32),1)'))
weather.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    55.0|    41.0| 4.7|drizzle|
|2012-01-02|         10.9|    51.1|    37.0| 4.5|   rain|
|2012-01-03|          0.8|    53.1|    45.0| 2.3|   rain|
|2012-01-04|         20.3|    54.0|    42.1| 4.7|   rain|
|2012-01-05|          1.3|    48.0|    37.0| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



    - Which month has the most rain, on average?

In [206]:
(
    weather.withColumn("month", sf.month("date"))
    .groupBy("month")
    .agg(sf.round(sf.mean("precipitation"),2).alias("avg_rainfall"))
    .sort(sf.col("avg_rainfall").desc())
    .first()
)

Row(month=11, avg_rainfall=5.35)

    - Which year was the windiest?

In [202]:
weather.show(1)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    55.0|    41.0| 4.7|drizzle|
+----------+-------------+--------+--------+----+-------+
only showing top 1 row



In [205]:
(
    weather.withColumn("year", sf.year("date"))
    .groupBy("year")
    .agg(sf.round(sf.mean("wind"),2).alias("avg_wind"))
    .sort(sf.col("avg_wind").desc())
    .first()
)

Row(year=2012, avg_wind=3.4)

    - What is the most frequent type of weather in January?

In [208]:
(
    weather.filter(sf.month("date") == 1)
    .groupBy("weather")
    .agg(sf.count("weather").alias('days'))
    .sort(sf.col('days').desc())
    .first()
)

Row(weather='fog', days=38)

    - What is the average high and low temperature on sunny days in July in 2013 and 2014?

In [158]:
weather.show(1)

+----------+-------------+----+-------+----------+----------+
|      date|precipitation|wind|weather|temp_max_f|temp_min_f|
+----------+-------------+----+-------+----------+----------+
|2012-01-01|          0.0| 4.7|drizzle|      55.0|      41.0|
+----------+-------------+----+-------+----------+----------+
only showing top 1 row



In [163]:
(
    weather.filter(sf.month("date") == 7)
    .filter(sf.year("date") == 2013)
    # .filter("weather" == "sun")
    .groupBy("weather")
    .agg(sf.mean("temp_max_f"), sf.mean("temp_min_f"))
    .show()
)

+-------+-----------------+-----------------+
|weather|  avg(temp_max_f)|  avg(temp_min_f)|
+-------+-----------------+-----------------+
|    fog|73.36666666666666|55.63333333333333|
|   rain|             72.0|             59.0|
|    sun|79.85555555555555|57.14444444444445|
+-------+-----------------+-----------------+



    - What percentage of days were rainy in q3 of 2015?

In [179]:
t = (
    weather.filter(sf.year("date") == 2015)
    .filter(sf.quarter("date") == 3)
    # .withColumn("rainy_day", weather == "rain")
    # .withColumn("quarter", sf.quarter("date"))
    # .withColumn("year", sf.year("date"))
    .groupBy("weather")
    .agg(((sf.count("weather")/90) * 100).alias("prct_weather"))
)
t.show()
# t.withColumn("prct_weather", days/90*100).show()

+-------+------------------+
|weather|      prct_weather|
+-------+------------------+
|    fog|23.333333333333332|
|drizzle| 5.555555555555555|
|    sun| 71.11111111111111|
|   rain|2.2222222222222223|
+-------+------------------+



    - For each year, find what percentage of days it rained (had non-zero precipitation).

In [210]:
(
    weather.withColumn("year", sf.year("date"))
    .filter(weather.precipitation > 0)
    .groupBy("year")
    .agg(sf.round(((sf.count("date")/365) * 100), 2).alias("prct_rainy"))
    .show()
)

+----+----------+
|year|prct_rainy|
+----+----------+
|2012|     48.49|
|2013|     41.64|
|2014|      41.1|
|2015|     39.45|
+----+----------+

